In [1]:
import numpy as np
import pandas as pd
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D
from keras.utils import np_utils, to_categorical
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
test_imagesKaggle = test.copy()
train_labelsKaggle = train['label']

print("train with Labels  :", train.shape)
print("train_labelsKaggle :", train_labelsKaggle.shape)

train.drop(['label'],axis=1, inplace=True)
train_imagesKaggle = train

print("train_imagesKaggle without Labels :", train_imagesKaggle.shape)
print("test_imagesKaggle  :", test_imagesKaggle.shape)

train with Labels  : (42000, 785)
train_labelsKaggle : (42000,)
train_imagesKaggle without Labels : (42000, 784)
test_imagesKaggle  : (28000, 784)


In [4]:
train_imagesKaggle = np.array(train_imagesKaggle).reshape(42000, 28, 28, 1).astype('float32') / 255
test_imagesKaggle = np.array(test_imagesKaggle).reshape(28000, 28, 28, 1).astype('float32') / 255

print('train_imagesKaggle :', train_imagesKaggle.shape)
print('test_imagesKaggle :', test_imagesKaggle.shape)

train_labelsKaggle = to_categorical(train_labelsKaggle)
print('train_labelsKaggle :', train_labelsKaggle.shape)

train_imagesKaggle : (42000, 28, 28, 1)
test_imagesKaggle : (28000, 28, 28, 1)
train_labelsKaggle : (42000, 10)


In [5]:
(train_imagesKeras, train_labelsKeras), (test_imagesKeras, test_labelsKeras) = mnist.load_data()

In [6]:
train_imagesKeras = train_imagesKeras.reshape(60000, 28, 28, 1).astype('float32') / 255
test_imagesKeras = test_imagesKeras.reshape(10000, 28, 28, 1).astype('float32') / 255

print('train_imagesKeras :' , train_imagesKeras.shape)
print('train_labelsKeras :', train_labelsKeras.shape)
print('test_imagesKeras :', test_imagesKeras.shape)
print('test_labelsKeras :', test_labelsKeras.shape)

train_labelsKeras = to_categorical(train_labelsKeras)
test_labelsKeras =  to_categorical(test_labelsKeras)
print('train_labelsKeras :', train_labelsKeras.shape)
print('test_labelsKeras :', test_labelsKeras.shape)

train_imagesKeras : (60000, 28, 28, 1)
train_labelsKeras : (60000,)
test_imagesKeras : (10000, 28, 28, 1)
test_labelsKeras : (10000,)
train_labelsKeras : (60000, 10)
test_labelsKeras : (10000, 10)


In [7]:
train_images = np.concatenate((train_imagesKeras, train_imagesKaggle), axis=0)
print('Train_images :', train_images.shape)

train_labels = np.concatenate((train_labelsKeras, train_labelsKaggle), axis=0)
print('Train_labels :', train_labels.shape)

Train_images : (102000, 28, 28, 1)
Train_labels : (102000, 10)


In [8]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [10]:
train_images = np.concatenate((train_images, test_imagesKeras), axis=0)
print("train_images :", train_images.shape)

train_labels = np.concatenate((train_labels, test_labelsKeras), axis=0)
print("train_labels :", train_labels.shape)

train_images : (112000, 28, 28, 1)
train_labels : (112000, 10)


In [26]:
history = model.fit(train_images, train_labels, epochs=3 ,batch_size=2048).history

Epoch 1/3
112000/112000 [==============================] - 2s 18us/step - loss: 2.9127e-04 - acc: 1.0000
Epoch 2/3
112000/112000 [==============================] - 2s 17us/step - loss: 9.6654e-04 - acc: 0.9998
Epoch 3/3
112000/112000 [==============================] - 2s 18us/step - loss: 2.9502e-04 - acc: 1.0000


In [27]:
raw_pred = model.predict(test_imagesKaggle)
pred_list = []
numTest = raw_pred.shape[0]

for i in range(numTest):
    pred_list.append(np.argmax(raw_pred[i]))

pred = np.array(pred_list)

In [28]:
sub = pd.read_csv('data/sample_submission.csv')

result = pd.DataFrame({'ImageId' : sub.ImageId, 'Label' : pred})
result.to_csv('sub.csv', index=False)